In [1]:
NAME = "Batsal Ghimire"

# Indexing Techniques and Data Structures.

## Counting Bloom Filters
### Bloom Filters
Bloom Filter is a very efficient data strucutre that can tell us if an item is present in a set or not. It has a constant time complexity for query and insertion, O(k), where 'k' is the number of hash functions. Bloom Filters also do not store any data, so it is incredibly memory efficient. Although it sounds like an ideal data strucutre, the price we pay for efficiency is that it is probabilistic in nature and we have a risk of getting a false positive.

Bloom Filters can only be used to find if we've seen an item before, and it gives us no information about what data it contains.

### How it works?
Let's say that Bloom Filter uses 'm' bits to contain 'n' items using 'k' hash functions. We need to make sure that the hash functions are independent to one another, lack of which might lead to more false positives and clustering. It's necessary to strike a balance between the number of items to be stored and the number of hash functions. This is because more hash functions makes the Bloom Filter slow, but too few and false positive rate increases.

Let's look at the diagram below to see how Bloom Filters Work.

<img src="fig1.png" alt="Fig1" style="width: 700px;"/>

First, we create a memory size with 'm' bits. And, we pass each item through the hash function which gives us the index values. We flip these indexes to 1 or True. When we go to retrieve these elements, we pass the item through the hash functions and get the index values. And, if all indexes consist of 1's, we return that we've seen the element.
<img src="fig2.png" alt="Fig2" style="width: 700px;"/>

If there are more than one elements with the same index, we don't update it to 2 and so on. It remains at 1. Here, we might observe the problem with this method. If we pass an element which is not in the data, but the index we get through the hashing function are all 1's, then we will still return that an element exists.

<img src="fig3.png" alt="Fig3" style="width: 700px;"/>

However, there is no possibility for a false negative. It can never say that something does not exist, if it does. Since, we don't delete the 1's, once it is established in place, it stays there.

Another problem with this is that it does not facilitate any way of deletion. If we delete the ones and replace it with zeros, then it might also affect the 1 implemented by another item. To solve this problem, there is an extension of bloom filters called 'Counting Bloom Filters.'

### CBF's
Counting Bloom filters also accept multiple bit counts, so we can also delete the items from the set. The array bits are incremented or decremented by 1 everytime we get the array position to be true. 

<img src="fig4.png" alt="Fig4" style="width: 700px;"/>

Since CBF's increase the count as more positions are True, we can decrement the counts by 1 to undo the increment caused by the item. This would be equivalent to deleting the item. This does not solve the problem of getting false positives, but it adds one more operation to the bloom filter. Also, we can change the threshold for the  CBF to only get True if the number of counts increases a certain point.

It still maintains the constant time complexity of the Bloom Filters for all operations: insertion, query, and deletion.

#### Selecting the optimal number of hash functions and memory size:
The Wikipedia articles on Bloom Filters mention the equation given below, which I have used for the Python implementation.

$$ k = \frac{m}{n} log (2)$$
where, 'k' is the number of hash functions, 'm' is the memory size, and 'n' is the number of elements that is to be inserted.

There is another equation to find the  appropriate memory size for the filter which is given as:

$$ m = \frac{-n log(p)}{(log(2))^2} $$
where, 'p' is the false positive probability.

From this equation, we can observe that we can change the false positive rate, depending on the use scenario and the room for error present.

### Applications
Counting Bloomm Filter's can be used in many areas, but we need to make sure that the consequence for a false positive are not very high.

One application can be to check if an important item exists before cleaning a drive or some storage device. It would guarantee to void the process if the item is still present and notify the user.

It can also help in the process of finding unique SSN, specially when there are millions of unique IDs already. We can generate a random number and check if that number already exists. Since, there is no chance of getting a false negative, getting a False would mean that the ID has never been used before.

Recommending videos on YouTube and Netflix, finding unique URLs, etc. can also make use of CBFs.

However, it should not be used in big missions where the missing of one file can be devastating (like space missions). If we ask a CBF to see if a file exists, it might return a true even if the file does not exist.


```python
txt_file = open("t8.shakespeare.txt", "r")

entries = txt_file.read().split(' ')
lines = [string.replace('\n', '') for string in entries]
all_text = [line for line in lines if line != '']
```

### For Hashing
We will be using mmh3 to generate the hash functions since it is very fast and we can make it very secure by changing the parameters.

mmh3 takes in two arguments. The first is the element which we intend to hash, and the second is the seed, if we choose to use it. In this implementation, I will be using the seeds which will be generated by the random module in python. This is the main reason I chose mmh3, since it can provide us with independent hash functions by just changing the seeds.

The hash function must be deterministic. So, the seed method in the random module will output a pseudo random number. This means that using the same seed should output the same number which maintains the deterministic nature.

### For Threshold
This implementation uses threshold, but it is not necessary since we can just return True if there is a non-zero value in the position.

In [2]:
#Import the libraries
import math
import random
import hashlib
import mmh3
# Feel free to define additional classes that you think are helpful 
# in building this class of CountingBloomFilter 

class CountingBloomFilter(object):
    """Implement the counting bloom filter which supports:
    - search: queries the membership of an element
    - insert: inserts a string to the filter
    - delete: removes a string from the filter 
    
    Feel free to define any helpful additional methods.
    """
    #We pass in the false positive rate and the number of items that needs be stored as the parameter. This is because the optimal number of hash functions and the array size can be computed by using the formula if we  have these parameters.
    def __init__(self, fpr, num_item, seed, threshold):
        """
        /YOUR ARGUMENTS/ are the two parameters of your choice from the 
        following parameters of a CBF:
        - fpr: float, false positive rate
        - memory_size: int, memory size
        - num_item: int, number of items stored
        - num_hashfn: int, number of hash functions
        
        For example, if you choose fpr and memory_size, edit your __init__ to
        `def __init__(self, memory_size, fpr)`
        """
        self.threshold = threshold
        self.seed = seed
        self.num_item = num_item
        self.fpr = fpr #I'm assuming that  fpr is given as a probability. If not, we can easily convert percentage into probability.
        
        #We will use the formula that we found above to find the size of the array and the number of hash functions.
        #First for memory_size, because we need it to find the num_hashfn.
        #Formula: m = -n ln(P)/(ln2)^2
        self.memory_size = abs(int(round(-self.num_item * math.log(self.fpr))/(math.log(2)**2))) #For this we need the math library
    
        #Now, for number of hash functions
        #Formula: k = (m/n)ln(2)
        self.num_hashfn = abs(int(round((self.memory_size/self.num_item)* math.log(2))))
        
        #We have the size of the list, so we can create the list and initialize it with all zeros.
        self.slots = self.memory_size * [0]
        
    def hash_cbf(self, item):
        """
        Returns hash values of an item
        [ADD ADDITIONAL DESCRIPTION, IF NEED BE]
        """
        #We use the mmh3 hash to get the hash function. mmh3 has two arguments, the first is the item which is fixed, but we differe the seed.
        for i in self.seed[:self.num_hashfn]:
            self.hash_val = mmh3.hash(item, i) % self.memory_size
        return self
    
    def search(self, item):
        """
        Searches for the item
        """
        #Seed should give us the same values as long as the initial state are the same.
        #We check if the slot value is greater than the threshold. For example: if a slot has value '2', then it compares it with the threshold we have set and return true if it is greater.
        for seed_val in self.seed[:self.num_hashfn]:
            if (self.slots[mmh3.hash(item, seed_val)%self.memory_size]) > self.threshold: #For this Assignment, I have set all the threshold to be 0.
                pass
            else:
                return False
        return True
    
    def insert(self, item):
        """
        Insert an item
        """
        for seed_val in self.seed[:self.num_hashfn]:
            #We get the index from the hash function, and we increase that position's value by 1.
            self.slots[mmh3.hash(item, seed_val)%self.memory_size] = (self.slots[mmh3.hash(item, seed_val)%self.memory_size]) + 1
        return self
    
    def delete(self, item):
        """
        Deletes an item
        """
        #Searches if the item exists. If not, there is no way to delete it.
        if self.search(item) == True:
            for seed_val in self.seed[:self.num_hashfn]:
                #Similar to when inserting, the value is incremented by 1. While deleting, we decrease this value by 1.
                self.slots[mmh3.hash(item, seed_val)%self.memory_size] = (self.slots[mmh3.hash(item, seed_val)%self.memory_size]) - 1
        return self


In [3]:
#This is a simple test used to demonstrate how the implementation works.
l = ['joker'] #We add one element to the list.
seed_list = [3,4,5,6,7,1] #Set the seeds.
cbf = CountingBloomFilter(0.4,2, seed_list,0) #Create an object with the given parameters
for i in l: # Goes through the list 'l'.
    print (i)
    cbf.insert(i) #Inserts the value in the set
cbf.search('joker') #Searches for joker, and we it returns True.

joker


True

In [4]:
#Given in the assignment.
txt_file = open("t8.shakespeare.txt", "r")

entries = txt_file.read().split(' ')
lines = [string.replace('\n', '') for string in entries]
all_text = [line for line in lines if line != '']
count = 0
l = []
#If we see that the implementation returns false, even if the element exists, it is because words with punctuation act as a single item.
"""These are the parameters we can adjust to get different results."""

#In the text file, in which word do you want the search to begin. 
start_word = 1826

#In the text file, in which word do you want the search to end. 
end_word = 1930

#Set the probability of getting a false positive
false_positive_prob = 0.2

#Set the number of seeds you want to use
num_seeds = 100

#Set the threshold you want to use.
threshold = 0

#What word would you like to search for,
search_word = 'give?'

#Which word would you like to delete.
del_word = ''

"""These are the parameters we can adjust to get different results."""

#This is just an unnecessary addition. Since the runtime was very high when I used the entire dataset, I implemented a way to select the start and end position of the word.
for i in range(start_word,end_word):
    l.append(all_text[i]) #Creates a list depending on the start and end values we enter
    count = count + 1
    
seeds = [] #Creates a set for the seeds.

for i in range(num_seeds): 
    seeds.append(random.randrange(0,100)) #Creates a list of seeds that we need to feed while creating the object
cbf = CountingBloomFilter(false_positive_prob,count,seeds,threshold) #We can change all these parameters.

for j in l: #Inserts each element from the list 'l' to the set.
    cbf.insert(j)
    
if del_word:
    if (cbf.search(del_word)) == True: #If it finds the word
        print ("Word was found.")
        print ("Deleting the word")
        cbf.delete(del_word) #Deletes the word
        if cbf.search(del_word) == False: #If the item is deleted and the search comes out as False
            print ("Your word '", del_word, "' is deleted.")
            print ("---------------------------------------")
        else: #If the item is deleted but the search comes out as True i.e. a false positive.
            print ("Something's not right, I don't feel too good.")
            print ("---------------------------------------------")
    else: #If the word does not exist in the first place
        print ("The word was not found.")
        print ("------------------------")
        
if search_word: #Searches for the word.
    if (cbf.search(search_word)) == True: #If it exists
         print ("The word '" , search_word , "'exists within the given range of words :)")
    else: #If  it does not exist.
         print ("The word '" , search_word , "'cannot be found.")

FileNotFoundError: [Errno 2] No such file or directory: 't8.shakespeare.txt'

### 1. Memory size and FPR
Theoretically, if we look at the equation, we can see that the relation between memory size(m) and FPR (probability) is inverse logarithmic. The graph we got also looks like the flipped version of the log graph. This shows that as the size of the array increases, the false positive rate decreases. This is due to the lessening number of collisions.

In [ ]:
#Building a random word generator
import random
import string
import sys
import matplotlib.pyplot as plt

#Adapted from: https://pynative.com/python-generate-random-string/
#The above given article implements a way to easily create a list of random words for testing.
def random_word(length):
    let = string.ascii_lowercase
    return ''.join(random.choice(let) for i in range(length))

false_positive = [] #Since, we are looking at the relation between memory size and FPR, we change the value of FPR.
for i in range(1,101):
    false_positive.append(i/100) #Change the percentage to probability, since this is what our class takes
false_positive_percentage = [round(100*j) for j in false_positive] #This is only used for plotting


num_item = 100 #Number of items we want to enter
items = [random_word(10) for words in range(num_item)] #Creates the given number of random words of length 10.
memory_size = [] #Sets the list  for memory size, which we use for plotting.
for k in false_positive: #Creates objects with varying FPR's
    cbf = CountingBloomFilter(k, num_item, seeds, 0)
    for i in items:
        cbf.insert(i) #Inserts the item
    memory_size.append(cbf.memory_size*sys.getsizeof(1)) #I found a way to find the memory_size with the help of https://stackoverflow.com/questions/449560/how-do-i-determine-the-size-of-an-object-in-python

#Rest of the segment is for plotting purposes.
plt.plot(false_positive_percentage, memory_size)
plt.xlabel("False Positive Rate (FPR) in percentage")
plt.ylabel("Memory size")
plt.grid(True)
plt.title("How memory size scales with FPR")
plt.show()

### Memory size and number of items
Again, like the linear relation between m and n in the equation, our graph is also linear. As more elements are added, the memory size also increases.

In [ ]:
#A lot is similar between the previous plot and this one. I will only comment the ones that are different.
num_item = 1000
items = [random_word(10) for words in range(num_item)]
memory_size = []
nums= 0
list_inputs = []
for num in range(1, num_item): #Changes the  number of items in the list
    n = items[:num] #Slices the list to get the items within the range
    cbf = CountingBloomFilter(0.1, len(n), seeds, 0)
    nums = nums + 1
    list_inputs.append(nums)
    for i in n:
        cbf.insert(i)
    memory_size.append(cbf.memory_size * sys.getsizeof(1))
plt.plot(list_inputs, memory_size)
plt.xlabel("Number of items")
plt.ylabel("Memory size")
plt.grid(True)
plt.title("How memory size scales with number of items")
plt.show()

### FPR and the number of hash functions
As we have more independent hash functions, less collision occurs because there are more conditions to be fulfilled before we return the presence or absence of the item.

In [ ]:
#Theoretical rate
false_positive = []
for i in range(1,101):
    false_positive.append(i/100)
false_positive_percentage = [round(100*j) for j in false_positive]
list_of_hashfn = []

for j in false_positive:
    cbf = CountingBloomFilter(j, len(items), seeds, 0)
    list_of_hashfn.append(cbf.num_hashfn)

#The way I do this is by creating two lists with random words. Then, I insert the items from the first list into the set. And, then I search for item that are not contained in the list 1 but still return as True i.e. False Positive.
list1 = [random_word(10) for words in range(500)]
list2 = [random_word(10) for words in range(500)]
li_fpr = []
t_hashfn = []


for k in false_positive:
    cbf = CountingBloomFilter(k, len(list1), seeds, 0)
    t_hashfn.append(cbf.num_hashfn)
    for li in list1:
        cbf.insert(li)
    n_false_positives = 0
    for v in list2:
        cbf.search(v)
        if cbf.search(v) == True and (v not in list1):
            n_false_positives += 1
    li_fpr.append(n_false_positives/len(list2))            
    
plt.plot(list_of_hashfn, false_positive)
plt.xlabel("Number of hash functions")
plt.ylabel("False positive probability")
plt.grid(True)
plt.title("Theoretical comparison")
plt.show()

plt.plot(t_hashfn, li_fpr)
plt.xlabel("Number of hash functions")
plt.ylabel("Fa")
plt.grid(True)
plt.title("Practical comparison")
plt.show()

### Access time and number of items
How many hash functions we use should not depend on the number of items to be stored. So, the access time is relatively constant with plenty of noise in both directions. Searching for some elements might take longer than other, which might explain this behavior.

In [ ]:
#I have used the time module to calculate the time and statistics to make it easy to find the average.
import time
import statistics

access_time = []
fpr = 0.05 #Set the FPR to be fixed
list_ = [random_word(10) for words in range(1000)]
size = []
for i in range(1, len(list_)):
    slic = list_[:i] #Slice the list like we did before
    cbf = CountingBloomFilter(fpr, len(slic), seeds, 0)
    for j in slic:
        cbf.insert(j)
        avg_time = []
    size.append(i)
        
    for i in range(10): #How many iterations we want to get more robust results.
        time_ = [] 
           
        for j in slic:
            begin_time = time.time()
            cbf.search(j)
            end_time = time.time()
            time_.append(end_time-begin_time)
        avg_time.append(statistics.mean(time_))
    access_time.append(statistics.mean(avg_time)) 

plt.plot(size, access_time)
plt.xlabel("Number of items")
plt.ylabel("Time to search for items")
plt.title("Relation between access time and number of items")
plt.show()